In [90]:
import pandas as pd
import pycountry_convert as pc
from pycountry import countries

## Model
suicide ~ population \
suicide ~ population + gdp_per_capita \
group by country (or year) (or age) suicide ~ population + gdp_per_capita

In [130]:
suicide_df = pd.read_csv('data/suicide.csv')
suicide_df = suicide_df.rename(columns={'suicides_no':'suicide', ' gdp_for_year ($) ':'total_gdp', 'gdp_per_capita ($)':'gdp_per_capita'})
suicide_df = suicide_df.drop(columns=['suicides/100k pop', 'HDI for year', 'country-year', 'gdp_per_capita', 'generation'])
suicide_df['total_gdp'] = suicide_df['total_gdp'].apply(lambda x: int(x.replace(',', '')))
suicide_df

,country,year,sex,age,suicide,population,total_gdp
0,Albania,1987,male,15-24 years,21,312900,2156624900
1,Albania,1987,male,35-54 years,16,308000,2156624900
2,Albania,1987,female,15-24 years,14,289700,2156624900
3,Albania,1987,male,75+ years,1,21800,2156624900
4,Albania,1987,male,25-34 years,9,274300,2156624900
...,...,...,...,...,...,...,...
27815,Uzbekistan,2014,female,35-54 years,107,3620833,63067077179
27816,Uzbekistan,2014,female,75+ years,9,348465,63067077179
27817,Uzbekistan,2014,male,5-14 years,60,2762158,63067077179
27818,Uzbekistan,2014,female,5-14 years,44,2631600,63067077179


## Get the rows belong to European countries

In [131]:
# Get the country names since some countries are not recognized by pycountry
countries_name_list = [country.name for country in list(countries)]

# Function to extract continent code
def get_continent(country):
    country_code = pc.country_name_to_country_alpha2(country)
    return pc.country_alpha2_to_continent_code(country_code)

suicide_df = suicide_df[suicide_df['country'].isin(countries_name_list)]   # Filter out invalid countries
suicide_df['continent'] = suicide_df['country'].apply(get_continent)       # Extract continent code
suicide_df = suicide_df[suicide_df['continent'] == 'EU']                   # Get the countries in EU
suicide_df.head(5)

C:\Users\Duong\AppData\Local\Temp\ipykernel_4748\1392511374.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  suicide_df['continent'] = suicide_df['country'].apply(get_continent)       # Extract continent code


,country,year,sex,age,suicide,population,total_gdp,continent
0,Albania,1987,male,15-24 years,21,312900,2156624900,EU
1,Albania,1987,male,35-54 years,16,308000,2156624900,EU
2,Albania,1987,female,15-24 years,14,289700,2156624900,EU
3,Albania,1987,male,75+ years,1,21800,2156624900,EU
4,Albania,1987,male,25-34 years,9,274300,2156624900,EU


## Group data by year and sex

In [139]:
# suicide_df = suicide_df.groupby(['country']).aggregate('sum').head(10)

suicide_df_grouped = suicide_df.groupby(['year', 'sex', 'country']).aggregate('sum')
suicide_df_grouped

C:\Users\Duong\AppData\Local\Temp\ipykernel_4748\2864376843.py:3: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  suicide_df_grouped = suicide_df.groupby(['year', 'sex', 'country']).aggregate('sum')


suicide  population      total_gdp
year sex    country                                        
1985 female Austria          624     3751216   416320646850
            Belgium          759     4759700   520380232758
            Bulgaria         422     4231000   102932526318
            France          3603    26422100  3318830486202
            Greece           128     4729100   286925105850
...                          ...         ...            ...
2016 male   Iceland           36      135122   101520490505
            Lithuania        693     1104112   213865149175
            Netherlands     1273     7017286  3886137707905
            Romania         1643     8054969   939029611745
            Sweden           782     4041436  2572299864030

[1852 rows x 3 columns]